# Đồ án cuối kỳ

Thông tin nhóm:
    
Phạm Quang Phước Nguyên - MSSV: 1612441

Nguyễn Cao Nhân - MSSV: 1612451

---
## Đặt vấn đề

---
## Parse trang web để lấy dữ liệu

In [1]:
from requests_html import HTMLSession
import requests
import time # Dùng để sleep chương trình
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

Mở kết nối

In [2]:
session = HTMLSession()

Hàm lấy từ điển URL dẫn đến kết quả tìm kiếm nhà trọ của từng quận (huyện)

In [3]:
def getDistricURLs(url):
    r = session.get(url).html
    estate_url_by_distric = {}
    
    districts = r.find(".district", first=True).find("a")
    for distric in districts:
        distric_url = distric.attrs["href"]
        #print(distric.text)
        estate_url_by_distric[distric.text] = distric_url 
        
    return estate_url_by_distric

Hàm lấy danh sách URL của các nhà trọ từ kết quả tìm kiếm

In [4]:
def getEstateURLs(url):
    estate_info_urls = []
    stop = False
    while url:
        r = session.get(url).html
        #print(url)     

        estate_list = r.find(".RealEstate_listing", first=True)
        estate_items = estate_list.find(".RealEstate_item")    

        for estate_item in estate_items:
            tilte = estate_item.find(".link", first=True)
            item_url = tilte.attrs["href"]
            estate_info_urls.append(item_url)
            #print(item_url) 

        url = ""
        pages = r.find(".pagination", first=True)
        if pages:
            next_page = pages.find("li")[-1].find("a", first=True)
            if next_page:
                url = next_page.attrs["href"]

        #time.sleep(0.2)
        
    return estate_info_urls

Định nghĩa các thuộc tính cần lấy

In [5]:
prop_list = ['address', 'price', 'size', 'exprire_data', 'update_date', 'object', 'sex', 'amenities', 'electric_price', 'water_price', 'internet_price', 'payment_method', 'close_time']#, 'detail']

prop_dict = {'Giới tính':'sex', 'Giá điện':'electric_price', 'Giá nước':'water_price', 'Giá wifi':'internet_price', 'Hình thức thanh toán':'payment_method', 'Giờ đóng cửa':'close_time'}

Hàm lấy thông tin nhà trọ từ trang thông tin chi tiết

In [6]:
def getEstateInfo(estate_info_url):
    r = session.get(estate_info_url).html
    
    info = {}
    info['url'] = estate_info_url

    info['address'] = r.find(".pr_location", first=True).find(".address", first=True).text

    pr_price_acreage = r.find(".pr_price_acreage", first=True)
    info['price'] = pr_price_acreage.find(".price", first=True).text
    size = pr_price_acreage.find(".acreage", first=True).text
    info['size'] = re.findall(r"\d+", size)[0]

    update_at = pr_price_acreage.find(".update_at")
    for date in update_at:
        text = date.text
        if text.startswith("Cập nhật"):
            info['update_date'] = re.findall(r"\d{2}/\d{2}/\d{4}", text)[0]
        elif text.startswith("Ngày hết hạn"):
            info['exprire_data'] = re.findall(r"\d{2}/\d{2}/\d{4}", text)[0]  

    highlight = r.find(".RealEstate_Highlight", first=True).find(".item")
    for item in highlight:
        name = item.find(".name", first=True).text
        value = item.find(".value", first=True).text
        prop_name = prop_dict.get(name)
        if prop_name:
            info[prop_name] = value
        elif name == 'Đối tượng':
            info['object'] = value.split(", ")

    amenities_list = r.find(".pr_amenities_list", first=True)    
    if amenities_list:
        amenities_list = amenities_list.find(".pr_amenitie_item")
        info['amenities'] = [amenity.text for amenity in amenities_list]

    info['detail'] = r.find(".RealEstate_Content", first=True).find(".content", first=True).text

    return info        

Hàm parse HTML từ các URL trong danh sách vào file csv

In [7]:
def parse_HTML_to_CSV(estate_info_urls, file_name):
    #open dataset file    
    print("Parsing {} HTMLs to {} ...".format(len(estate_info_urls), file_name))
    dataset_file = open(file_name, "w", encoding='utf-8')

    #write header
    dataset_file.write("url")
    for prop in prop_list:
        dataset_file.write("\t " + prop)
    dataset_file.write("\n")

    #parse HTML 
    k = 0
    current_percent = 0    
    for estate_info_url in estate_info_urls:    
        estate_info = getEstateInfo(estate_info_url)

        #write record   
        dataset_file.write(estate_info['url'])    
        for prop in prop_list:
            cell = estate_info.get(prop, "")
            dataset_file.write("\t " + str(cell))
        dataset_file.write("\n")
        
        #show progress        
        k+=1
        percent = int(k * 100/ len(estate_info_urls))
        if percent > current_percent and percent % 25 == 0:
            current_percent = percent
            print("Finished {} % ...".format(percent))

    #save dataset
    dataset_file.close()

    print("Finished writing to {}".format(file_name))

---
## Ghi dữ liệu parse được vào file csv

Cờ hiệu cho biết đã có dataset hay chưa

In [8]:
HAS_DATASET = False

Lấy thông tin của các nhà trọ ở quận 5

In [9]:
if HAS_DATASET == False:
    url = "https://bds123.vn/cho-thue-phong-tro-nha-tro-ho-chi-minh.html"

    distric_urls = getDistricURLs(url)
    print(distric_urls.keys())

    q5_urls = getEstateURLs(distric_urls['Quận 5'])

    parse_HTML_to_CSV(q5_urls, "nhatro_q5.csv")

dict_keys(['Quận 1', 'Quận 2', 'Quận 3', 'Quận 4', 'Quận 5', 'Quận 6', 'Quận 7', 'Quận 8', 'Quận 9', 'Quận 10', 'Quận 11', 'Quận 12', 'Bình Thạnh', 'Bình Tân', 'Gò Vấp', 'Phú Nhuận', 'Tân Bình', 'Tân Phú', 'Thủ Đức', 'Bình Chánh', 'Cần Giờ', 'Củ Chi', 'Hóc Môn', 'Nhà Bè'])
Parsing 150 HTMLs to nhatro_q5.csv ...
Finished 25 % ...
Finished 50 % ...
Finished 75 % ...
Finished 100 % ...
Finished writing to nhatro_q5.csv


---
## Đọc dữ liệu từ file csv vào dataframe

In [10]:
import numpy as np
%matplotlib inline
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [11]:
df = pd.read_csv("nhatro_q5.csv", sep = "\t")

In [12]:
df.head(1)

,url,address,price,size,exprire_data,update_date,object,sex,amenities,electric_price,water_price,internet_price,payment_method,close_time
0,https://bds123.vn/phong-tro-gia-re-cho-sinh-vi...,"77/1A Đường Nguyễn Thi, Phường 13, Quận 5, Hồ...",1.3 triệu/tháng,10,02/12/2019,30/11/2019,['Sinh viên'],,"['Internet tốc độ cao', 'Bếp']",,60.000 đồng/người,70.000 đồng,Thỏa thuận,


---
## Mô tả dữ liệu

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 14 columns):
url                150 non-null object
 address           150 non-null object
 price             150 non-null object
 size              150 non-null int64
 exprire_data      150 non-null object
 update_date       150 non-null object
 object            150 non-null object
 sex               150 non-null object
 amenities         150 non-null object
 electric_price    150 non-null object
 water_price       150 non-null object
 internet_price    150 non-null object
 payment_method    150 non-null object
 close_time        150 non-null object
dtypes: int64(1), object(13)
memory usage: 16.5+ KB


Tập dữ liệu nhà trọ cho thuê được trích lấy từ trang web https://bds123.vn/. Dữ liệu chứa một số thông tin trong các mẫu tin đăng cho thuê nhà trọ (giới hạn trong khu vực Quận 5, Thành phố HCM). Số mẫu dữ liệu nhóm thu thập là 150, số thuộc tính là 14, cụ thể như sau:
<ol>
    <li>url: đường dẫn tới mẫu tin</li>
    <li>address: địa chỉ chi tiết của nhà trọ</li>
    <li>price: giá cho thuê</li>
    <li>size: tổng diện tích của phòng trọ / nhà trọ</li>
    <li>exprire_data: ngày hết hạn của mẫu tin</li>
    <li>update_date: ngày cập nhật mới nhất của mẫu tin (có thể không phải là ngày đầu tiên đăng tin)</li>
    <li>object: đối tượng được cho thuê (ví dụ: nhân viên văn phòng, người đi làm, sinh viên, ... )</li>
    <li>sex: giới tính đối tượng được cho thuê </li>
    <li>amenities: các tiện nghi đi kèm: tủ lạnh, máy giặt, gần công viên, ban công, nhà bếp, bảo vệ, ... </li>
    <li>electric_price: tiền điện</li>
    <li>water_price: tiền nước</li>
    <li>internet_price: tiền wifi</li>
    <li>payment_method: phương thức thanh toán</li>
    <li>close_time: thời gian đóng cửa nhà trọ vào ban đêm </li>
</ol>
Tập dữ liệu thu được ban đầu là dữ liệu thô chưa được chuẩn hóa, một số mẫu bị thiếu giá trị ở một số thuộc tính. Địa chỉ nhà trọ chưa được tách ra thành thuộc tính đường, phường; các tiện nghi còn gom chung vào một thuộc tính, tiền điện và tiền nước có những đơn giá, cách tính khác nhau. Giai đoạn tiền xử lý dữ liệu phía sau sẽ thực hiện các thao tác để chuẩn hóa, xử lý lại dữ liệu ban đầu thu được.



---
## Tiền xử lí dữ liệu

---
## Trực quan hóa dữ liệu

---
## Train mô hình

---
## Test mô hình

---
## Đánh giá kết quả

---
## Kết luận